## Data preprocessing
Let's first load the packages that will be necessary for the analysis and download the data from Nanostring's webiste.

In [1]:
import anndata as ad
import pandas as pd
import scanpy as sc
import squidpy as sq
import numpy as np
import os
import matplotlib.pyplot as plt

/Users/ljiang/anaconda3/envs/cellcharter_env_py11/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/ljiang/anaconda3/envs/cellcharter_env_py11/lib/python3.11/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


In [2]:
int_dir = "/Users/ljiang/Partners HealthCare Dropbox/FilbinLab/projects/Ependymoma/12 - Xenium/h5ad_files/"
out_dir = "analysis/adata_obj/"

In [3]:
adata = sc.read_h5ad(int_dir + 'combined_adata_unprocessed.h5ad')

/Users/ljiang/anaconda3/envs/cellcharter_env_py11/lib/python3.11/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [4]:
adata.obs_names_make_unique()
adata.obs = adata.obs.rename_axis('index')

In [5]:
adata.obs['SampleName'] = pd.Categorical(adata.obs['SampleName'])
adata

AnnData object with n_obs × n_vars = 5915996 × 366
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region', 'cell_labels', 'Metaprogram', 'SampleName', 'SampleID'
    obsm: 'spatial'

In [6]:
sc.pp.filter_genes(adata, min_counts=3)
sc.pp.filter_cells(adata, min_counts=3)

In [7]:
adata.layers["counts"] = adata.X.copy()

sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

In [8]:
adata.obs['cell_barcode'] = adata.obs['cell_id']
adata.obs.drop('cell_id', axis=1, inplace=True)

In [9]:
adata

AnnData object with n_obs × n_vars = 5914092 × 366
    obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'region', 'cell_labels', 'Metaprogram', 'SampleName', 'SampleID', 'n_counts', 'cell_barcode'
    var: 'n_counts'
    uns: 'log1p'
    obsm: 'spatial'
    layers: 'counts'

In [ ]:
adata.write(out_dir + "adata.h5ad", compression="gzip")